In [74]:
import pandas as pd 

In [75]:
df = pd.read_csv("inputs/prezenta_vot.csv")
df_jud = pd.read_csv("inputs/Coduri_Judete.csv")
df_regiuni = pd.read_csv("inputs/Coduri_Regiuni.csv")

### 1_EXTRAGEREA LOCALITATILOR CARE AU AVUT PREZENTA LA VOT > 50%
- setul de date rezultat trebuie sa aiba structura cod_localitate - nume_localitate - prezenta_vot


- se extrage coloanele de localitate, de LP(lista permanenta de vot) si de LT(voturile totale)
- se transforma valorile in procente si se salveaza listele noi rezultate 
- se aplica o masca pentru a selecta doar localitatiile cu prezenta la vot > 50%
- se scriu in outputs/part1.csv

In [76]:
df_prezenta_procentuala_vot = df.loc[:, ['Siruta','Localitate', 'Votanti_LP', 'LT']]
df_prezenta_procentuala_vot['LT'] = df_prezenta_procentuala_vot['LT']/df_prezenta_procentuala_vot['Votanti_LP'] * 100
percentage_constraint_mask = df_prezenta_procentuala_vot['LT'] > 50 
df_prezenta_procentuala_vot[percentage_constraint_mask].to_csv('outputs/part1.csv', columns=['Siruta', 'Localitate', 'LT'])
print(df_prezenta_procentuala_vot[percentage_constraint_mask])

      Siruta     Localitate  Votanti_LP         LT
1       1071         Ciugud        2656  53.765060
12      2381       Arieseni        1366  66.398243
20      3805         Cergau        1276  50.705329
21      3841  Ceru-Bacainti         213  77.934272
23      4008     Ciuruleasa         945  61.904762
...      ...            ...         ...        ...
3134  179604  Poienile Izei         724  50.276243
3152  179793         Buesti         834  55.875300
3156  179837           Coas        1179  55.470738
3165  179926       Rascaeti        1789  58.188932
3176  180037         Boghis        1471  51.801496

[359 rows x 4 columns]


### 2_SORTAREA LOCALITATILOR IN FUNCTIE DE PREZENTA PROCENTUALA LA VOT
- se sorteaza localitatiile dupa prezenta procentuala la vot si se scriu in acelasi format ca si in partea1 intr-un fisier .csv

In [77]:
df_prezenta_procentuala_vot.sort_values(by=['LT'], inplace=True, ascending=False)
print(df_prezenta_procentuala_vot)
df_prezenta_procentuala_vot.to_csv('outputs/part2.csv', columns=['Siruta', 'Localitate', 'LT'])

      Siruta    Localitate  Votanti_LP          LT
1879  111220        Ciresu         382  147.120419
1471   89151        Cerbal         391  120.971867
1910  113153  Poroina Mare         762  113.385827
1905  112806        Padina         964  105.082988
1901  112600      Livezile        1146  104.450262
...      ...           ...         ...         ...
3179  180064    Barbulesti        4097   11.398584
2507  145998        Vurpar        2086   10.882071
1633   98916      Scanteia        9128   10.319895
1619   98202         Mosna        7510    7.723036
1602   97241      Grajduri        8245    7.713766

[3181 rows x 4 columns]


### 3_EXTRAGEREA NIVELULUI DE PREZENTA LA VOT PER REGIUNI
- cum in df(prezenta_vot.csv) nu avem date despre regiuni va trebui sa facem merge cu df_judete unde avem si regiunile
- creaza un dataframe nou care sa retina totalul de votanti_lp/regiune si totalul LT/regiune 
- tranformam in procente 
- salvam in fisier .csv

In [78]:
df_slice = df.loc[:, ['Siruta', 'Judet', 'Votanti_LP', 'LT']]
#print(df_slice)
#print(df_jud)
df_regional = df_slice.merge(df_jud, left_on='Judet', right_on='IndicativJudet')
print(df_regional)

df_percentage_per_region = pd.DataFrame()
df_percentage_per_region['Region'] = df_regional['Regiune'].unique()

for currentRegion in df_percentage_per_region['Region'].values:
    region_mask = df_regional['Regiune'] == currentRegion
    current_region_sum = df_regional[region_mask]['Votanti_LP'].sum()
    current_region_LT = df_regional[region_mask]['LT'].sum()
    #df.loc[current_idx, 'Votanti_totali']
    df_percentage_per_region.loc[df_percentage_per_region['Region'] == currentRegion, 'Votanti_LP_per_region'] = current_region_sum
    df_percentage_per_region.loc[df_percentage_per_region['Region'] == currentRegion, 'LT_per_region'] = current_region_LT

df_percentage_per_region['LT_per_region_percentage'] = df_percentage_per_region['LT_per_region']/df_percentage_per_region['Votanti_LP_per_region'] * 100

df_percentage_per_region.to_csv('outputs/part3.csv')

print(df_percentage_per_region)

      Siruta Judet  Votanti_LP     LT IndicativJudet  NumeJudet    Regiune
0       1017    ab       61753  21764             ab       Alba     Centru
1       1071    ab        2656   1428             ab       Alba     Centru
2       1151    ab        4491   1645             ab       Alba     Centru
3       1213    ab       21348   5949             ab       Alba     Centru
4       1348    ab       16807   5623             ab       Alba     Centru
...      ...   ...         ...    ...            ...        ...        ...
3176  180037    sj        1471    762             sj      Salaj  Nord-Vest
3177  180046    il        2043    517             il   Ialomita        Sud
3178  180055    cl        1653    606             cl   Calarasi        Sud
3179  180064    il        4097    467             il   Ialomita        Sud
3180  180091    sm        2438    839             sm  Satu Mare  Nord-Vest

[3181 rows x 7 columns]
            Region  Votanti_LP_per_region  LT_per_region  \
0           Cen

### 4_EXTRAGEREA PER LOCALITATE A CATEGORIEI DE VARSTA PREDOMINANT PREZENTA LA VOT
- se creaza un dataframe nou ce tine suma dintre nr de femei si barbati per categorii de varsta
- se adauga dataframe-ului o coloana cu varsta ce retine numele coloanei de varsta cu cea mai numeroasa prezenta
- rezultatul se va scrie intr-un fisier .csv 

In [80]:
df_age_groups_segregated = df.iloc[:, df.columns.get_loc('Barbati_18-24'):]
df_age_groups_united = pd.DataFrame()
df_age_groups_united['Localitate'] = df['Localitate']
df_age_groups_united['18-24'] = df_age_groups_segregated['Barbati_18-24'] + df_age_groups_segregated['Femei_18-24']
df_age_groups_united['25-34'] = df_age_groups_segregated['Barbati_25-34'] + df_age_groups_segregated['Femei_25-34']
df_age_groups_united['35-44'] = df_age_groups_segregated['Barbati_35-44'] + df_age_groups_segregated['Femei_35-44']
df_age_groups_united['45-64'] = df_age_groups_segregated['Barbati_45-64'] + df_age_groups_segregated['Femei_45-64']
df_age_groups_united['65_'] = df_age_groups_segregated['Barbati_65_'] + df_age_groups_segregated['Femei_65_']    

df_age_groups_united['Dominant_age_group'] = df_age_groups_united.iloc[:, 1:].idxmax(axis=1)

df_age_groups_united.to_csv('outputs/part4.csv')

print(df_age_groups_united)


                 Localitate  18-24  25-34  35-44  45-64   65_  \
0     Municipiul Alba Iulia   1623   3016   4531   8490  4104   
1                    Ciugud    104    211    262    500   351   
2                Oras Abrud    137    166    232    800   310   
3           Municipiul Aiud    488    650    859   2508  1444   
4           Municipiul Blaj    461    617    894   2261  1390   
...                     ...    ...    ...    ...    ...   ...   
3176                 Boghis     77    125    133    264   163   
3177         Gura Ialomitei     47     54     93    204   119   
3178                 Crivat     39     78     89    250   150   
3179             Barbulesti     58     73    102    198    36   
3180                  Racsa     50     76    119    352   242   

     Dominant_age_group  
0                 45-64  
1                 45-64  
2                 45-64  
3                 45-64  
4                 45-64  
...                 ...  
3176              45-64  
3177       

### 5_SA SE EXTRAGA DOAR ACELE LOCALITATI CARE AU CA MAJORITATE GRUPA DOMINANTA DE VARSTA
- pentru a gasi grupa majoritara de varsta putem sa calculam modul pe setul coloana dominant_age_group din cerinta trecuta
- dupa ce am gasit grupa majoritara vom face o masca pe setul de date care o cuprinde doar pe ea 
- scriem datele gasite in csv